In [13]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"

def make_deltas(masks):
    deltas = []
    for h in range(len(masks)):
        if h == 0:
            deltas.append([1 for _ in range(masks.shape[1])])
        else:
            deltas.append([1 for _ in range(masks.shape[1])] + (1-masks[h]) * deltas[-1])
    
    return list(deltas)


class MyDataset(Dataset):
    def __init__(self, dataset, q):
        self.data = dataset
        self.q = q

    def __len__(self):
        return self.data.shape[1] // self.q

    def __getitem__(self, index):
        return self.data[:,index * self.q : index * self.q + self.q,:]

class MyDataset2(Dataset):
    def __init__(self, dataset, q):
        self.data = dataset
        self.q = q

    def __len__(self):
        return self.data.shape[1] // self.q

    def __getitem__(self, index):
        return self.data[:,index * self.q : index * self.q + self.q,:]

def missing_data_rbf(df,rbf, batch_size, seq_len):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0)
    
    mydata  = MyDataset(dataset, seq_len)
    data = DataLoader(mydata, batch_size, shuffle=False)

    return data


def val_missing_data_rbf(df, real, rbf, batch_size, seq_len):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    
    masks = ~np.isnan(values)
    real_masks = ~np.isnan(real.values)
    
    masks = masks.reshape(shp)
    real_masks = real_masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    real = torch.nan_to_num(torch.from_numpy(real.values).to(torch.float32))

    masks = torch.from_numpy(masks).to(torch.float32)
    real_masks = torch.from_numpy(real_masks).to(torch.float32)
    
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)


    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0), real.unsqueeze_(0), real_masks.unsqueeze_(0)], dim = 0)
    mydata  = MyDataset2(dataset, seq_len)
    data = DataLoader(mydata, batch_size, shuffle=False)
    
    return data

def eval_model(model, rbf, realpath, dfpath, batch_size, seq_len):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)

    dataset = val_missing_data_rbf(df,real, rbf, batch_size, seq_len)

    model.eval()
    c_list = []
    train_list = []
    real_list = []
    real_masks_list = []
    masks_list = []

    batch_x_loss = 0.0
    # batch_loss = 0.0
    # batch_d_loss = 0.0

    for data in dataset:
        data = data.to(device)
        value = data[:,0,:].clone().detach()
        real_value = data[:,4,:].clone().detach()
        real_masks = data[:,5,:].clone().detach()
        train_masks = data[:,2,:].clone().detach()

        train_list.append(value)
        real_list.append(real_value)
        masks_list.append(train_masks)
        real_masks_list.append(real_masks)

        c_hat_list, imputations, x_loss = model(data)
        c_list.append(c_hat_list)
        batch_x_loss += x_loss
        # batch_d_loss += D_loss
        # batch_loss = loss

    c_list = torch.cat(c_list, dim = 0)
    c_list_ori = c_list * torch.tensor(df.std().values, device= device).reshape(1,1,36) + torch.tensor(df.mean().values, device= device).reshape(1,1,36)

    real_list = torch.cat(real_list, dim = 0) # none scale
    real_masks_list = torch.cat(real_masks_list, dim = 0)

    masks_list = torch.cat(masks_list, dim = 0)
    train_list = torch.cat(train_list, dim = 0)

    missing_masks = real_masks_list - masks_list
    # print("Scale MAE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    # print("Scale MRE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", torch.sum(torch.abs((c_list_ori - real_list) * missing_masks)) / torch.sum(missing_masks))
    print("Original MRE :", torch.sum(torch.abs((c_list_ori - real_list) * missing_masks)) / torch.sum(torch.abs(real_list * missing_masks)))
    print()
    print('Train MAE :', torch.sum(torch.abs((c_list_ori - real_list) * masks_list)) / torch.sum(masks_list))
    print("Train MRE :", torch.sum(torch.abs((c_list_ori - real_list) * masks_list)) / torch.sum(torch.abs(real_list * masks_list)))
    print('--------------------------------------------------------------')


    print("x_loss: {}".format(batch_x_loss))

def val_missing_data_rbf2(df,rbf):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0).unsqueeze_(0)

    return dataset

def eval_model2(model, rbf, realpath, dfpath):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    dataset = val_missing_data_rbf2(df,rbf)
    dataset = dataset.to(device)

    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)
    real_scaler = (real - df.mean()) / df.std()

    df_scaler = ((df-df.mean()) / df.std()).values
    masks = ~np.isnan(df_scaler)
    masks = torch.from_numpy(masks).to(torch.float32)
    
    eval_masks = ~np.isnan(real_scaler.values)
    eval_masks = torch.from_numpy(eval_masks).to(torch.float32)

    test_masks = eval_masks - masks
    real_scaler = torch.nan_to_num(torch.from_numpy(real_scaler.values).to(torch.float32))
    
    model.eval()
    c_hat_list, imputations, x_los = model(dataset)

    Nonscale_imputataion = pd.DataFrame(c_hat_list[0].cpu().detach() , columns= df.columns)
    Nonscale_imputataion = (Nonscale_imputataion * df.std()) + df.mean()
    
    real = real.fillna(0)
    print("Scale MAE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    print("Scale MRE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(test_masks.cpu().numpy()))
    print("Original MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(np.abs(real.values * test_masks.cpu().numpy())))

    print('Train MAE :', np.sum(np.abs((Nonscale_imputataion - real).values * masks.cpu().numpy())) / np.sum(masks.cpu().numpy()))
    print("Train MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * masks.cpu().numpy())) / np.sum(np.abs(real.values * masks.cpu().numpy())))

    return Nonscale_imputataion

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from tqdm import tqdm
import math
import random
import numpy as np
import pandas as pd
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
class FeatureRegression(nn.Module):
    def __init__(self, input_size):
        super(FeatureRegression, self).__init__()
        self.build(input_size)

    def build(self, input_size):
        self.W = Parameter(torch.Tensor(input_size, input_size))
        self.b = Parameter(torch.Tensor(input_size))

        m = torch.ones(input_size, input_size).cuda() - torch.eye(input_size, input_size).cuda()
        self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, x):
        z_h = F.linear(x, self.W * Variable(self.m), self.b)
        return z_h

class TemporalDecay(nn.Module):
    def __init__(self, input_size, output_size, diag = False):
        super(TemporalDecay, self).__init__()
        self.diag = diag

        self.build(input_size, output_size)

    def build(self, input_size, output_size):
        self.W = Parameter(torch.Tensor(output_size, input_size)).cuda()
        self.b = Parameter(torch.Tensor(output_size)).cuda()
        self.relu = nn.ReLU(inplace=False)
        if self.diag == True:
            assert(input_size == output_size)
            m = torch.eye(input_size, input_size).cuda()
            self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, d):
        gamma = self.relu(F.linear(d, self.W, self.b))
        gamma = torch.exp(-gamma)
        return gamma

class MGRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MGRU, self).__init__()

        self.temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.temp_decay_x = TemporalDecay(input_size, input_size, diag = True)
        self.temp_decay_r = TemporalDecay(input_size, input_size, diag = True)
        
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.build()

    def build(self):
        self.output_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        
        self.z_layer = FeatureRegression(self.input_size)
        self.beta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.lstmcell = nn.LSTMCell(self.input_size * 2, self.hidden_size)
        self.concat_lyaer = nn.Linear(self.input_size * 2, self.input_size)
        

    def loss(self, hat, y, m):
        return torch.sum(torch.abs((y - hat)) * m) / (torch.sum(m) + 1e-5)

    
    def forward(self, input):
        values = input[:,0,::]
        delta = input[:,1,::]
        masks = input[:,2,::]
        rbfs = input[:,3,::]

        hid = torch.zeros((values.size(0), self.hidden_size)).cuda()
        c = torch.zeros((values.size(0), self.hidden_size)).cuda()

        x_loss = 0.0
        imputations = []
        c_hat_list = []
        for i in range(values.size(1)):

            v = values[:,i,:]
            d = delta[:,i,:]
            m = masks[:,i,:]
            r = rbfs[:,i,:]

            gamma_x = self.temp_decay_x(d)
            gamma_h = self.temp_decay_h(d)
            
            hid = hid * gamma_h

            r_hat = self.temp_decay_r(r)
            
            x_hat = self.output_layer(hid)
            x_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)

            RG = torch.cat([x_hat, r_hat], dim = 1)
            concat_hat = self.concat_lyaer(RG)
            x_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)

            x_c = m * v + (1 - m) * x_hat

            z_hat = self.z_layer(x_c)
            x_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)

            beta_weight = torch.cat([gamma_x, m], dim = 1)
            beta = torch.sigmoid(self.beta_layer(beta_weight))

            c_hat = beta * z_hat + (1 - beta) * x_hat
            x_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)

            c_c = m * v + (1 - m) * c_hat

            gru_input = torch.cat([c_c, m], dim = 1)
            imputations.append(c_c.unsqueeze(dim = 1))
            c_hat_list.append(c_hat.unsqueeze(1))
            
            # GRU cell
            hid, c = self.lstmcell(gru_input, (hid, c))

        c_hat_list = torch.cat(c_hat_list, dim = 1)
        imputations = torch.cat(imputations, dim = 1)
        return c_hat_list, imputations, x_loss

In [9]:
def train_MGRU(model, lr, epochs, dataset, device):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    progress = tqdm(range(epochs))
    
    imputation_list = []
    loss_list = []
    c_hat_list2 = []
    model.to(device)

    for epoch in progress:
        batch_loss = 0.0
        for data in dataset:
            data = data.to(device)
            c_hat_list, imputations, x_loss = model(data)
        
            optimizer.zero_grad()
            x_loss.backward()
            optimizer.step()
            
            batch_loss += x_loss
        progress.set_description("loss: {}".format(batch_loss))

    return x_loss
    

In [15]:
# dropout None and x_loss non scale # target : real
dfpath = 'pm25_missing.txt'
df = pd.read_csv("./dataset/"+dfpath).drop(["datetime"], axis = 1)
rbfpath = "air_20_8.0_scale.csv"
batch_size = 64
dataset = missing_data_rbf(df, rbfpath, batch_size, 36)

model = MGRU(36, 64)
model.to(device)
loss_list = train_MGRU(model, 0.001, 1000, dataset, device)
Nonscale_imputataion = eval_model(model, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('-------------------------------------------------------')
Nonscale_imputataion2 = eval_model2(model, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 80.20755004882812: 100%|██████████| 1000/1000 [09:26<00:00,  1.77it/s]


Original MAE : tensor(15.7806, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2218, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(10.0546, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.1186, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 80.18170166015625
-------------------------------------------------------
Scale MAE : tensor(0.1785)
Scale MRE : tensor(0.2578)
Original MAE : 14.363976087027106
Original MRE : 0.2017685469775867
Train MAE : 9.648453223618784
Train MRE : 0.11381291967402266


In [14]:
Nonscale_imputataion = eval_model(model, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('-------------------------------------------------------')
Nonscale_imputataion2 = eval_model2(model, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

Original MAE : tensor(15.2823, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2148, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(9.9346, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.1172, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 79.3597640991211
-------------------------------------------------------
Scale MAE : tensor(0.1764)
Scale MRE : tensor(0.2546)
Original MAE : 14.228894161937726
Original MRE : 0.19987107210133465
Train MAE : 9.497607523178598
Train MRE : 0.11203354745865618


In [16]:
# dropout None and x_loss non scale # target : real
dfpath = 'pm25_missing.txt'
df = pd.read_csv("./dataset/"+dfpath).drop(["datetime"], axis = 1)
rbfpath = "air_20_8.0_scale.csv"
batch_size = 64
dataset = missing_data_rbf(df, rbfpath, batch_size, 36)

model = MGRU(36, 64)
model.to(device)
loss_list = train_MGRU(model, 0.001, 1000, dataset, device)
Nonscale_imputataion = eval_model(model, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('-------------------------------------------------------')
Nonscale_imputataion2 = eval_model2(model, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 80.33904266357422: 100%|██████████| 1000/1000 [09:12<00:00,  1.81it/s]


Original MAE : tensor(15.0759, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2119, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(10.0590, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.1187, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 80.08767700195312
-------------------------------------------------------
Scale MAE : tensor(0.1762)
Scale MRE : tensor(0.2544)
Original MAE : 14.114621901308514
Original MRE : 0.19826590735813907
Train MAE : 9.639329556298367
Train MRE : 0.11370529711610539


In [18]:
class MGRU2(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MGRU2, self).__init__()

        self.temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.temp_decay_x = TemporalDecay(input_size, input_size, diag = True)
        self.temp_decay_r = TemporalDecay(input_size, input_size, diag = True)
        
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.build()

    def build(self):
        self.output_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        
        self.z_layer = FeatureRegression(self.input_size)
        self.beta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.grucell = nn.GRUCell(self.input_size * 2, self.hidden_size)
        self.concat_lyaer = nn.Linear(self.input_size * 2, self.input_size)
        

    def loss(self, hat, y, m):
        return torch.sum(torch.abs((y - hat)) * m) / (torch.sum(m) + 1e-5)

    
    def forward(self, input):
        values = input[:,0,::]
        delta = input[:,1,::]
        masks = input[:,2,::]
        rbfs = input[:,3,::]

        hid = torch.zeros((values.size(0), self.hidden_size)).cuda()

        x_loss = 0.0
        imputations = []
        c_hat_list = []
        for i in range(values.size(1)):

            v = values[:,i,:]
            d = delta[:,i,:]
            m = masks[:,i,:]
            r = rbfs[:,i,:]

            gamma_x = self.temp_decay_x(d)
            gamma_h = self.temp_decay_h(d)
            
            hid = hid * gamma_h
            
            x_hat = self.output_layer(hid)
            x_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)

            RG = torch.cat([x_hat, r], dim = 1)
            concat_hat = self.concat_lyaer(RG)
            x_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)

            x_c = m * v + (1 - m) * x_hat

            z_hat = self.z_layer(x_c)
            x_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)

            beta_weight = torch.cat([gamma_x, m], dim = 1)
            beta = self.beta_layer(beta_weight)

            c_hat = beta * z_hat + (1 - beta) * x_hat
            x_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)

            c_c = m * v + (1 - m) * c_hat

            gru_input = torch.cat([c_c, m], dim = 1)
            imputations.append(c_c.unsqueeze(dim = 1))
            c_hat_list.append(c_hat.unsqueeze(1))
            
            # GRU cell
            hid = self.grucell(gru_input, hid)

        c_hat_list = torch.cat(c_hat_list, dim = 1)
        imputations = torch.cat(imputations, dim = 1)
        return c_hat_list, imputations, x_loss

In [19]:
# sigmoid 제거 버전
dfpath = 'pm25_missing.txt'
df = pd.read_csv("./dataset/"+dfpath).drop(["datetime"], axis = 1)
rbfpath = "air_20_8.0_scale.csv"
batch_size = 64
dataset = missing_data_rbf(df, rbfpath, batch_size, 36)

model8 = MGRU2(36, 64)
model8.to(device)
loss_list = train_MGRU(model8, 0.001, 1000, dataset, device)
Nonscale_imputataion = eval_model(model8, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2(model8, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 74.18373107910156: 100%|██████████| 1000/1000 [08:31<00:00,  1.95it/s]


Original MAE : tensor(16.1102, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2265, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(9.8333, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.1160, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 73.90876007080078
------------------------------------------------------
Scale MAE : tensor(0.1816)
Scale MRE : tensor(0.2621)
Original MAE : 14.553447092370268
Original MRE : 0.20443001683877643
Train MAE : 9.39867295126559
Train MRE : 0.1108665176534481


In [20]:
# sigmoid 제거 버전
dfpath = 'pm25_missing.txt'
df = pd.read_csv("./dataset/"+dfpath).drop(["datetime"], axis = 1)
rbfpath = "air_20_8.0_scale.csv"
batch_size = 64
dataset = missing_data_rbf(df, rbfpath, batch_size, 36)

model8 = MGRU2(36, 64)
model8.to(device)
loss_list = train_MGRU(model8, 0.001, 1000, dataset, device)
Nonscale_imputataion = eval_model(model8, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2(model8, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 74.35720825195312: 100%|██████████| 1000/1000 [08:38<00:00,  1.93it/s]


Original MAE : tensor(15.6611, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2202, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(9.8732, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.1165, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 74.17981719970703
------------------------------------------------------
Scale MAE : tensor(0.1680)
Scale MRE : tensor(0.2425)
Original MAE : 13.59862229132166
Original MRE : 0.19101774076991435
Train MAE : 9.451292665903004
Train MRE : 0.11148721852813731


In [21]:
# sigmoid 제거 버전
dfpath = 'pm25_missing.txt'
df = pd.read_csv("./dataset/"+dfpath).drop(["datetime"], axis = 1)
rbfpath = "air_20_8.0_scale.csv"
batch_size = 64
dataset2 = missing_data_rbf(df, rbfpath, batch_size, 36)

model8 = MGRU2(36, 64)
model8.to(device)
loss_list = train_MGRU(model8, 0.001, 1000, dataset2, device)
Nonscale_imputataion = eval_model(model8, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2(model8, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 74.32839965820312: 100%|██████████| 1000/1000 [08:24<00:00,  1.98it/s]


Original MAE : tensor(15.8912, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2234, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(9.9067, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.1169, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 74.32269287109375
------------------------------------------------------
Scale MAE : tensor(0.1823)
Scale MRE : tensor(0.2632)
Original MAE : 14.649843599060604
Original MRE : 0.20578408363551737
Train MAE : 9.446119565248738
Train MRE : 0.11142619675857653
